# Exploring conformational space of selected macrocycles - "M13"

In this notebook we present and analyze selected structures, technical notes are [here](www.gitlab.com/user/gosia/icho).

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Fri Apr  7 13:10:24 2017


In [3]:
# Functions used in this notebook:

def grep_energies_from_sdf_outputs(files):
    energies = {}
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies[os.path.splitext(os.path.basename(inp))[0]] = float(lines[i+1])
    return energies

def find_duplicates(rms_sorted, energy):
    i = 0
    to_be_deleted = []
    while i < len(rms_sorted):
        j = i + 1
        while j < len(rms_sorted):
            if rms_sorted[i][0] in to_be_deleted:
                i = i + 1
                j = j + 1
            elif rms_sorted[j][0] in to_be_deleted:
                j = j + 1
            else:
                rms1 = rms_sorted[i][1]
                rms2 = rms_sorted[j][1]
                if (rms2 - rms1) < rms_thresh:
                    if energy[rms_sorted[i][0]] < energy[rms_sorted[j][0]]:
                        to_be_deleted.append(rms_sorted[j][0])
                    else:
                        to_be_deleted.append(rms_sorted[i][0])
                else:
                    break
        i = i + 1
    if to_be_deleted:
        print("Conformers which will be deleted:")    
        print(to_be_deleted)
    return to_be_deleted

## Crystal structure of "M13" macrocycle

In [4]:
cm13 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m13/m13_crystal.xyz','r').read()
vcm13 = py3Dmol.view(width=400,height=400)
vcm13.removeAllModels()
vcm13.addModel(cm13,'xyz')
vcm13.setStyle({'stick':{'radius':0.15,'color':'spectrum'}})
vcm13.setBackgroundColor('0xeeeeee')
vcm13.zoomTo()
vcm13.show()

In [6]:
# "core" is a part of a molecule, which we wish to be the "most-aligned" among multiple conformers
smiles      = 'N1C(=O)c2cc(C(=O)NCCC(C)CCNC(=O)c3cc(C(=O)NCCCCC1)cc(c3)C(C)(C)C)cc(c2)C(C)(C)C'
core_smiles = 'c1ccccc1'

m13 = Chem.AddHs(Chem.MolFromSmiles(smiles))
core_m13 = m13.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))

templ_m13 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_crystal.sdf')
m13_crystal = templ_m13[0]

## Conformers generated with the Balloon software:

Conformers were generated in two ways (genetic algorithm):

* Starting with the crystal geometry kept as a template, output: "m13_b_crystal" on the left fig. below

* Starting with the SMILES signature of M1 and allowing to "rebuild the geometry" (option --rebuildGeometry), output: "m13_b_smiles" on the right fig. below

In both cases the Balloon software was asked for 50 conformers.

In [7]:
inps_m13_b_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_crystalsdf/*.sdf')

In [8]:
inps_m13_b_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/results_starting_from_crystalsmiles/*.sdf')

In [9]:
e_m13_b_sdf = grep_energies_from_sdf_outputs(inps_m13_b_sdf)
e_m13_b_smi = grep_energies_from_sdf_outputs(inps_m13_b_smi)

In [10]:
%%html
<table>
  <tr>
    <td id="m13_b_crystal" ></td>
    <td id="m13_b_smiles"  ></td>
  <tr>
    <td> m13_b_crystal </td>
    <td> m13_b_smiles  </td>  
  </tr>
</table>

,
m13_b_crystal,m13_b_smiles


In [11]:
# write conformers to dictionaries
allmol_m13_b_sdf = {}
allmol_m13_b_smi = {}
suppl_m13_b_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_crystal_sdfout.sdf')
suppl_m13_b_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/balloon/m13_crystal_smilesout.sdf')

for i, mol in enumerate(suppl_m13_b_sdf):
    name = "m13_b_sdf_" + str(i)
    allmol_m13_b_sdf[name] = mol
for i, mol in enumerate(suppl_m13_b_smi):
    name = "m13_b_smi_" + str(i)
    allmol_m13_b_smi[name] = mol    

In [12]:
# align:
for key, mol in allmol_m13_b_sdf.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
for key, mol in allmol_m13_b_smi.items():   
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))    

In [13]:
# view:
p13_b_handles=[]

p13_b_sdf = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_b_sdf.items():
    mb = Chem.MolToMolBlock(mol)
    p13_b_sdf.addModel(mb,'sdf')
p13_b_sdf.setStyle({'stick':{'radius':'0.15'}})
p13_b_sdf.setBackgroundColor('0xeeeeee')
p13_b_sdf.zoomTo()    
p13_b_handles.append(p13_b_sdf)

p13_b_smi = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_b_smi.items():
    mb = Chem.MolToMolBlock(mol)
    p13_b_smi.addModel(mb,'sdf')
p13_b_smi.setStyle({'stick':{'radius':'0.15'}})
p13_b_smi.setBackgroundColor('0xeeeeee')
p13_b_smi.zoomTo()    
p13_b_handles.append(p13_b_smi)

In [14]:
p13_b_handles[0].insert('m13_b_crystal')

In [15]:
p13_b_handles[1].insert('m13_b_smiles')

### pre-screening

Some of the generated conformers are very much alike. To remove potential duplicates which were not "caught" by the Balloon program, we can compare the energies (preoptimized with MM) and the RMSD calculated against a reference structure (here: the crystal structure of M13). It does not matter against which structure we are aligning the conformers, since we are interested in relative RMS between them.

First let's print the energies and RMS values:

In [16]:
allmol_m13_b = {}
allmol_m13_b.update(allmol_m13_b_sdf)
allmol_m13_b.update(allmol_m13_b_smi)

energy_m13_b = {}
energy_m13_b.update(e_m13_b_sdf)
energy_m13_b.update(e_m13_b_smi)

rms_m13_b = {}
for key, mol in allmol_m13_b.items():
    rms_m13_b[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m13_crystal))
    #print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m13_b[key], rms_m13_b[key]))

Then we can introduce some thresholds, for instance:

* if two conformers differ by less than 0.01 in RMS (measured against the reference structure), then select the one with the lower energy

In [17]:
rms_sorted = sorted(rms_m13_b.items(), key=lambda x: x[1])
rms_thresh = 0.01

print("List sorted by RMS:")
for i, t in enumerate(rms_sorted):
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(rms_sorted[i][0], energy_m13_b[rms_sorted[i][0]], rms_sorted[i][1]))

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m13_b)

for mol in to_be_deleted:
    del allmol_m13_b[mol]
    del energy_m13_b[mol]
    del rms_m13_b[mol]  

List sorted by RMS:
name = m13_b_sdf_1, E = 117.203377, RMS = 0.603265
name = m13_b_sdf_3, E = 117.827359, RMS = 1.234760
name = m13_b_smi_18, E = 129.334009, RMS = 1.365857
name = m13_b_sdf_2, E = 117.541962, RMS = 1.380316
name = m13_b_smi_24, E = 130.963754, RMS = 1.380595
name = m13_b_smi_3, E = 121.093257, RMS = 1.445001
name = m13_b_sdf_0, E = 117.035303, RMS = 1.466614
name = m13_b_smi_27, E = 131.655801, RMS = 1.537981
name = m13_b_sdf_11, E = 119.884262, RMS = 1.558754
name = m13_b_sdf_6, E = 119.030168, RMS = 1.563261
name = m13_b_sdf_4, E = 117.916466, RMS = 1.570615
name = m13_b_smi_31, E = 133.690811, RMS = 1.597154
name = m13_b_smi_28, E = 132.366842, RMS = 1.618309
name = m13_b_smi_29, E = 132.952990, RMS = 1.644996
name = m13_b_smi_6, E = 122.658644, RMS = 1.669362
name = m13_b_smi_8, E = 124.415569, RMS = 1.692369
name = m13_b_smi_25, E = 131.368688, RMS = 1.742208
name = m13_b_sdf_37, E = 122.701451, RMS = 1.765604
name = m13_b_smi_16, E = 126.447517, RMS = 1.768478
n

Below we will align the selected conformers:

In [18]:
for key, mol in allmol_m13_b.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
    
p_b = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_b.items():
    mb = Chem.MolToMolBlock(mol)
    p_b.addModel(mb,'sdf')
p_b.setStyle({'stick':{'radius':'0.15'}})
p_b.setBackgroundColor('0xeeeeee')
p_b.zoomTo()
p_b.show()

## Conformers generated with the RDKit software:

In [19]:
inps_m13_rdkit_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/results_crystal_from_smiles/*.sdf')
inps_m13_rdkit_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/results_crystal_from_sdf/*.sdf')

In [20]:
e_m13_rdkit_smi = grep_energies_from_sdf_outputs(inps_m13_rdkit_smi)
e_m13_rdkit_sdf = grep_energies_from_sdf_outputs(inps_m13_rdkit_sdf)

In [21]:
# write conformers to dictionaries
allmol_m13_rdkit_smi = {}
suppl_m13_rdkit_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/result_smiles.sdf')
allmol_m13_rdkit_sdf = {}
suppl_m13_rdkit_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m13/rdkit/result_sdf.sdf')

for i, mol in enumerate(suppl_m13_rdkit_smi):
    name = "m13_rdkit_smi_" + str(i)
    allmol_m13_rdkit_smi[name] = mol  
for i, mol in enumerate(suppl_m13_rdkit_sdf):
    name = "m13_rdkit_sdf_" + str(i)
    allmol_m13_rdkit_sdf[name] = mol     

In [22]:
# align:
for key, mol in allmol_m13_rdkit_smi.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
for key, mol in allmol_m13_rdkit_sdf.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles)) 
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))

In [23]:
%%html
<table>
  <tr>
    <td id="m13_rdkit_crystal" ></td>
    <td id="m13_rdkit_smiles"  ></td>
  <tr>
    <td> m13_rdkit_crystal </td>
    <td> m13_rdkit_smiles  </td>  
  </tr>
</table>

,
m13_rdkit_crystal,m13_rdkit_smiles


In [24]:
# view:
p13_rdkit_handles=[]

p13_rdkit_sdf = py3Dmol.view(width=400,height=400)
p13_rdkit_sdf.removeAllModels()
for key, mol in allmol_m13_rdkit_sdf.items(): 
    mb = Chem.MolToMolBlock(mol)
    p13_rdkit_sdf.addModel(mb,'sdf')    
p13_rdkit_sdf.setStyle({'stick':{'radius':'0.15'}})
p13_rdkit_sdf.setBackgroundColor('0xeeeeee')
p13_rdkit_sdf.zoomTo()
p13_rdkit_handles.append(p13_rdkit_sdf)

p13_rdkit_smi = py3Dmol.view(width=400,height=400)
p13_rdkit_smi.removeAllModels()
for key, mol in allmol_m13_rdkit_smi.items(): 
    mb = Chem.MolToMolBlock(mol)
    p13_rdkit_smi.addModel(mb,'sdf')    
p13_rdkit_smi.setStyle({'stick':{'radius':'0.15'}})
p13_rdkit_smi.setBackgroundColor('0xeeeeee')
p13_rdkit_smi.zoomTo()
p13_rdkit_handles.append(p13_rdkit_smi)

In [25]:
p13_rdkit_handles[0].insert('m13_rdkit_crystal')

In [26]:
p13_rdkit_handles[1].insert('m13_rdkit_smiles')

### pre-screening

In [28]:
allmol_m13_rdkit = {}
allmol_m13_rdkit.update(allmol_m13_rdkit_sdf)
allmol_m13_rdkit.update(allmol_m13_rdkit_smi)

energy_m13_rdkit = {}
energy_m13_rdkit.update(e_m13_rdkit_sdf)
energy_m13_rdkit.update(e_m13_rdkit_smi)

rms_m13_rdkit = {}
for key, mol in allmol_m13_rdkit.items():
    rms_m13_rdkit[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m13_crystal))
    #print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m13_rdkit[key], rms_m13_rdkit[key]))

In [29]:
rms_sorted = sorted(rms_m13_rdkit.items(), key=lambda x: x[1])
rms_thresh = 0.01

#print("List sorted by RMS:")
#for i, t in enumerate(rms_sorted):
#    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(rms_sorted[i][0], energy_m13_rdkit[rms_sorted[i][0]], rms_sorted[i][1]))

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m13_rdkit)

for mol in to_be_deleted:
    del allmol_m13_rdkit[mol]
    del energy_m13_rdkit[mol]
    del rms_m13_rdkit[mol]  

Conformers which will be deleted:
['m13_rdkit_smi_22', 'm13_rdkit_sdf_13', 'm13_rdkit_smi_28', 'm13_rdkit_smi_21', 'm13_rdkit_sdf_36', 'm13_rdkit_smi_34', 'm13_rdkit_smi_30', 'm13_rdkit_smi_39', 'm13_rdkit_sdf_38', 'm13_rdkit_smi_1', 'm13_rdkit_sdf_2', 'm13_rdkit_smi_2', 'm13_rdkit_smi_9', 'm13_rdkit_smi_12', 'm13_rdkit_smi_15', 'm13_rdkit_sdf_19', 'm13_rdkit_smi_11', 'm13_rdkit_sdf_22', 'm13_rdkit_smi_33', 'm13_rdkit_smi_25', 'm13_rdkit_sdf_44', 'm13_rdkit_sdf_9', 'm13_rdkit_smi_5', 'm13_rdkit_smi_7', 'm13_rdkit_sdf_34', 'm13_rdkit_smi_35', 'm13_rdkit_smi_49', 'm13_rdkit_smi_27', 'm13_rdkit_smi_46', 'm13_rdkit_smi_23', 'm13_rdkit_sdf_43', 'm13_rdkit_smi_32', 'm13_rdkit_smi_40', 'm13_rdkit_smi_31', 'm13_rdkit_sdf_39', 'm13_rdkit_smi_47', 'm13_rdkit_smi_20', 'm13_rdkit_smi_42', 'm13_rdkit_smi_19', 'm13_rdkit_sdf_32']


Below we will align the selected conformers:

In [30]:
for key, mol in allmol_m13_rdkit.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
    
p_r = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13_rdkit.items():
    mb = Chem.MolToMolBlock(mol)
    p_r.addModel(mb,'sdf')
p_r.setStyle({'stick':{'radius':'0.15'}})
p_r.setBackgroundColor('0xeeeeee')
p_r.zoomTo()
p_r.show()

## Summary

Now let's generate a list of all conformers (from all programs used, as presented above). We can further pre-screen all the structures and remove potential duplicates. Here we can also use more crude threshold.

In [32]:
allmol_m13 = {}
allmol_m13.update(allmol_m13_b)
allmol_m13.update(allmol_m13_rdkit)

energy_m13 = {}
energy_m13.update(energy_m13_b)
energy_m13.update(energy_m13_rdkit)

rms_m13 = {}
for key, mol in allmol_m13.items():
    rms_m13[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m13_crystal))
    #print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m13[key], rms_m13[key]))

In [33]:
rms_sorted = sorted(rms_m13.items(), key=lambda x: x[1])
rms_thresh = 0.01

# now compare RMS of each pair, if the structures are too similar then delete the one with the higher energy
to_be_deleted = find_duplicates(rms_sorted, energy_m13)

for mol in to_be_deleted:
    del allmol_m13[mol]
    del energy_m13[mol]
    del rms_m13[mol]  

Conformers which will be deleted:
['m13_b_smi_6', 'm13_b_sdf_7', 'm13_b_sdf_5', 'm13_b_smi_0', 'm13_b_sdf_10', 'm13_b_sdf_16', 'm13_b_sdf_14', 'm13_b_sdf_15', 'm13_b_smi_26', 'm13_b_sdf_9', 'm13_b_sdf_29', 'm13_b_smi_12', 'm13_b_sdf_25', 'm13_b_smi_21', 'm13_b_sdf_19', 'm13_b_sdf_23', 'm13_b_sdf_21', 'm13_b_sdf_27', 'm13_b_smi_4', 'm13_b_sdf_36', 'm13_b_sdf_24', 'm13_b_sdf_22', 'm13_b_sdf_28']


Finally we can align all conformers which will further be used as starting points in DFT geometry optimizations:

In [34]:
for key, mol in allmol_m13.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles(core_smiles))
    AllChem.AlignMol(mol,m13_crystal,atomMap=list(zip(core_mol,core_m13)))
    
p = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m13.items():
    mb = Chem.MolToMolBlock(mol)
    p.addModel(mb,'sdf')
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

Write the selected conformers' names to the list "list_selected_conformers_from_balloon_rdkit". It will be used to generate Gaussian inputs:

In [35]:
with open("/home/gosia/work/work_on_gitlab/icho/calcs/m13/list_selected_conformers_from_ballon_rdkit", "w") as f:
    for key, mol in allmol_m13.items():
        f.write(key+"\n")      